# 🌍 ESG & Financial Intelligence Platform
## AI-Powered Analysis of Corporate Reports (ESG And Annual)

**🎯 What this does:**
- Automatically processes PDF reports from Google Cloud Storage
- Uses Gemini 2.5 Pro to extract financial & ESG metrics
- Generates forecasts using Google's TimesFM 2.0 model
- Analyzes companies: Amgen, Novartis, Target

**📊 Features:**
- ✅ Automated PDF processing from Cloud Storage
- ✅ AI-powered data extraction (Financial + ESG metrics)
- ✅ Revenue forecasting with TimesFM 2.0
- ✅ Multi-company comparative analysis

---
**👆 Click "Runtime" → "Run all" to see the magic happen!**

# 🚀 Setup & Authentication

In [93]:

print("🌟 Welcome to the ESG & Financial Intelligence Platform!")
print("🔧 Setting up environment...")

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

# Install required packages
!pip install -q bigframes google-cloud-bigquery plotly seaborn

# Import libraries
import bigframes.pandas as bpd
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

print("✅ Authentication complete!")
print("✅ Libraries installed!")

🌟 Welcome to the ESG & Financial Intelligence Platform!
🔧 Setting up environment...
✅ Authentication complete!
✅ Libraries installed!


# Intializing Parameters

In [94]:
# Replace the below with other company names from this list:
# novartis, amgen, target, bankofmontreal, lloyd, wellsfargo

company1 = "amgen"
company2 ="wellsfargo"
company3 = "lloyd"

PROJECT_ID = "intellitrend-project-dev"   # update if needed
PROJECT_LOCATION="US" # Others are EU, ASIA
CLOUD_RES_CONN = "ghack_conn" #must be all small case


DATASET_ID = "db_reports_insights_annual_esg" #Name of dataset to be created in BQ
OBJ_TABLE_ID = "all_reports_obj_table_metadata" #Objec table name. That holds the metadata of the source files in GCS
RAW_TABLE_ID="all_reports_ai_text_raw"   # the output text that gets extracted based on prmpt - using ML.GENERATE_TEXT
RAW2_TABLE_ID="all_reports_ai_text_raw2" # the output text that gets extracted based on prmpt - using AI.GENERATE (An alternate way - for comparison)
CURATED_TABLE_ID="all_reports_ai_text_curated" # response_text extracted from JSON output
FINAL_TABLE_ID="all_reports_ai_text_final" # format the output from the curated table
FORECAST_TABLE_ID="all_reports_forecast" # forecast table from the formatted-output

QUALIFIED_CLOUD_RES_CONN = f"{PROJECT_ID}.{PROJECT_LOCATION}.{CLOUD_RES_CONN}"

QUALIFIED_OBJ_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{OBJ_TABLE_ID}"
QUALIFIED_RAW_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE_ID}"
QUALIFIED_RAW2_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{RAW2_TABLE_ID}"

QUALIFIED_CURATED_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{CURATED_TABLE_ID}"
QUALIFIED_FINAL_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE_ID}"
QUALIFIED_FORECAST_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{FORECAST_TABLE_ID}"

MODEL_ENDPOINT="gemini-2.5-pro"
MODEL_NAME="gemini_model_25pro"
QUALIFIED_MODEL_ID=f"{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}"

DATA_FILES_PATH="gs://report_insights"

# 🏗️ Project Configuration

In [95]:

from google.colab import userdata

# Configure BigFrames
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = PROJECT_LOCATION

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Project configured: {PROJECT_ID}")
print("✅ BigFrames ready for action!")

✅ Project configured: intellitrend-project-dev
✅ BigFrames ready for action!


#  Create Cloud Resource Connection

In [96]:
!bq mk \
  --connection \
  --project_id={PROJECT_ID} \
  --connection_type=CLOUD_RESOURCE \
  --location={PROJECT_LOCATION} \
  {CLOUD_RES_CONN}

BigQuery error in mk operation: Already Exists: Connection
projects/573553606303/locations/us/connections/ghack_conn


# Fetch Service Account associated with the Cloud Res Connection

In [97]:
SERVICE_ACCT = !bq show --format=prettyjson \
  --connection \
  --project_id={PROJECT_ID} \
  --location={PROJECT_LOCATION} \
  {CLOUD_RES_CONN} | grep "serviceAccountId" | cut -d '"' -f 4

SERVICE_ACCT_EMAIL = SERVICE_ACCT[0]  # first (and only) line
print(SERVICE_ACCT_EMAIL)

bqcx-573553606303-2s0n@gcp-sa-bigquery-condel.iam.gserviceaccount.com


# Give the below roles to the Conn Service Account:
  * objectViewer and  
  * aiplatformUser



In [98]:
import time

# Use $SERVICE_ACCT_EMAIL so the Python variable expands in the shell
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
    --role="roles/storage.objectViewer" \
    --format=none

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
    --role="roles/aiplatform.user" \
    --format=none

# Wait ~60 seconds for IAM updates to propagate
time.sleep(60)

Updated IAM policy for project [intellitrend-project-dev].
Updated IAM policy for project [intellitrend-project-dev].


# Create the Gemini Model

In [99]:
# 🤖 Step 0a: Create Gemini 2.5 Pro Model
print("🤖 Creating Gemini 2.5 Pro model...")

create_model_sql = f"""
CREATE OR REPLACE MODEL `{QUALIFIED_MODEL_ID}`
REMOTE WITH CONNECTION `{QUALIFIED_CLOUD_RES_CONN}`
OPTIONS (ENDPOINT = '{MODEL_ENDPOINT}');
"""

try:
    job = client.query(create_model_sql)
    job.result()
    print("🎉 Gemini 2.5 Pro model created successfully!")
except Exception as e:
    print(f"⚠️  Model creation issue: {e}")

🤖 Creating Gemini 2.5 Pro model...
🎉 Gemini 2.5 Pro model created successfully!


# 📁 Create Object Table for PDF Reports

In [100]:


print(f"📁 Creating external table(Object Table): '{OBJ_TABLE_ID}' for PDF reports...")

create_external_table_sql = f"""
CREATE OR REPLACE EXTERNAL TABLE `{QUALIFIED_OBJ_TABLE_ID}`
WITH CONNECTION `{QUALIFIED_CLOUD_RES_CONN}`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['{DATA_FILES_PATH}/*']
);
"""

try:
    job = client.query(create_external_table_sql)
    job.result()
    print("✅ External table created successfully!")

    # Check what files we have
    check_files_sql = f"SELECT uri, size FROM `{QUALIFIED_OBJ_TABLE_ID}`  "
    files_df = bpd.read_gbq(check_files_sql)
    print(f"📄 Found {len(files_df)} files in storage")
    print("Sample files:")
    print(files_df.head())

except Exception as e:
    print(f"⚠️  Note: {e}")
    print("   Make sure your GCS bucket and connection are set up correctly")

📁 Creating external table(Object Table): 'all_reports_obj_table_metadata' for PDF reports...
✅ External table created successfully!


📄 Found 79 files in storage
Sample files:


                                                 uri      size
0  gs://report_insights/wellsfargo/wellsfargo_sus...    642974
1  gs://report_insights/lloyd/lloyd_sustainabilit...  10223646
2  gs://report_insights/jpmorganchase/jpmorgancha...   7143472
3  gs://report_insights/amgen/amgen_annualreport_...   8154936
4  gs://report_insights/amgen/amgen_annualreport_...   3651904

[5 rows x 2 columns]


#  🧠 Extract Financial & ESG Data - Using ML.GENERATE_TEXT

In [102]:

print(f"🧠 Processing PDFs with {MODEL_NAME}...")
print(f"This analyzes {company1}, {company2}, and {company3} reports...")

generate_text_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_RAW_TABLE_ID}` AS (
SELECT
  *
FROM
  ML.GENERATE_TEXT(
    MODEL `{QUALIFIED_MODEL_ID}`,
    TABLE `{QUALIFIED_OBJ_TABLE_ID}`,
    STRUCT(
      '''
        You are an expert ESG and Financial analyst. Use only the information provided in the document to answer.\n
        Fetch Financial and Sustainability Details including tabular and image data for each PDF.\n
        Read the filename. If it is a sustainability report, please get only ESG details. If it is an annual report, get just the financials as below.\n
        Ensure you fetch details so it has answers for the below:\n
        Financial Performance\n
        What is the total revenue this year?\n
        How has revenue changed compared to last year?\n
        What is the net profit or loss?\n
        How have the earnings per share (EPS) changed?\n
        Are gross margins improving or declining?\n
        Are operating margins stable or volatile?\n
        How has net margin shifted year over year?\n
        Is cash flow from operations positive and consistent?\n
        How much free cash flow is available?\n
        How are working capital levels trending?\n\n
        Balance Sheet & Liquidity\n
        What is the total debt level?\n
        Is the debt-to-equity ratio rising or falling?\n
        Does the company have enough liquidity to cover short-term liabilities?\n
        What is the current ratio and quick ratio?\n
        How much cash and equivalents are available?\n
        Are interest expenses sustainable?\n
        How sensitive is the company to interest rate changes?\n
        What is the level of contingent liabilities?\n\n
        Business Strategy\n
        What are the company’s core growth initiatives?\n
        Is management investing in new markets or products?\n
        How much is spent on research and development (R&D)?\n
        Are acquisitions or divestitures planned?\n
        Is the company expanding geographically?\n
        What is the company’s competitive advantage?\n
        Is the advantage sustainable against competitors?\n
        Is digital transformation a core part of the strategy?\n\n
        Risk Factors\n
        What macroeconomic risks affect the business?\n
        What industry-specific risks are highlighted?\n
        What operational risks are disclosed?\n
        Are supply chain risks material?\n
        Are legal or regulatory risks mentioned?\n
        How is foreign exchange risk managed?\n
        How dependent is revenue on a few customers?\n
        Are raw material price risks significant?\n\n
        Governance & Leadership\n
        Is the board majority independent?\n
        How many women or minorities are on the board?\n
        Is executive compensation performance-linked?\n
        Does management own significant equity?\n
        Are shareholder rights well protected?\n
        Is succession planning disclosed?\n
        Has management delivered on past promises?\n
        Is the auditor independent and credible?\n\n
        Shareholder Value\n
        Is the company paying dividends?\n
        Is the dividend payout sustainable?\n
        Is the dividend policy clear?\n
        Are share buybacks taking place?\n
        Is return on equity (ROE) improving?\n
        Is return on assets (ROA) stable?\n
        Has shareholder equity grown consistently?\n
        What is the earnings guidance for next year?\n\n
        Investor Questions – Sustainability Reports\n\n
        Environmental\n
        What is the total Scope 1 emissions?\n
        What is the total Scope 2 emissions?\n
        What is the total Scope 3 emissions?\n
        Are science-based targets disclosed?\n
        Has the company committed to net-zero?\n
        What percentage of energy is renewable?\n
        Is energy efficiency improving year over year?\n
        What is the total water withdrawal?\n
        What is water recycled or reused?\n
        Is the company exposed to water stress?\n
        How much total waste is generated?\n
        How much waste is recycled or diverted from landfill?\n
        Are hazardous waste levels disclosed?\n
        How sustainable are raw material sourcing practices?\n
        Are biodiversity risks addressed?\n\n
        Social\n
        What is the total workforce size?\n
        What percentage of employees are women?\n
        What percentage of leadership roles are held by women?\n
        What is minority representation in the workforce?\n
        What is the employee turnover rate?\n
        How many workplace injuries occurred?\n
        What is the lost-time injury frequency rate?\n
        Are labor rights respected across the supply chain?\n
        Is child labor risk disclosed in supply chains?\n
        Are employee training hours increasing per year?\n
        Are wages above local living wage levels?\n
        Is there employee engagement survey data?\n
        What community development initiatives exist?\n
        What philanthropic contributions were made?\n
        Is there disclosure on data privacy and cybersecurity?\n\n
        Governance (ESG-specific)\n
        Is there a dedicated board ESG committee?\n
        How often does the board review ESG matters?\n
        Is ESG risk included in enterprise risk management?\n
        Is executive compensation tied to ESG goals?\n
        Are ESG targets linked to management incentives?\n
        Is ESG performance externally assured?\n
        Are ESG disclosures aligned with GRI or SASB?\n
        Does the company report under TCFD or ISSB?\n
        Is CDP reporting publicly available?\n
        Are stakeholder engagement practices disclosed?\n\n
        Risk & Opportunity\n
        What are the identified physical climate risks?\n
        What are the identified transition risks?\n
        Is there disclosure on carbon pricing exposure?\n
        How dependent is revenue on carbon-intensive activities?\n
        What sustainability-linked opportunities are identified?\n
        Are green products or services generating revenue?\n
        Is access to sustainable finance increasing?\n
        Are suppliers evaluated for ESG performance?\n
        Are ESG clauses included in supplier contracts?\n
        Is there disclosure on product lifecycle impacts?\n
      ''' AS prompt,

      0 AS temperature,
      8092 AS max_output_tokens
    )
  )
WHERE uri like '%{company1}%' or uri like '%{company2}%' or  uri like '%{company3}%'
);
"""

try:
    print("⏳ Processing... This may take 2-3 minutes")
    job = client.query(generate_text_sql)
    job.result()

    # Check results
    check_sql = f"SELECT COUNT(*) as processed_files FROM `{QUALIFIED_RAW_TABLE_ID}`"
    result = client.query(check_sql).result()
    count = list(result)[0][0]
    print(f"✅ Successfully processed {count} files!")

except Exception as e:
    print(f"⚠️  Processing issue: {e}")

🧠 Processing PDFs with gemini_model_25pro...
This analyzes amgen, wellsfargo, and lloyd reports...
⏳ Processing... This may take 2-3 minutes


KeyboardInterrupt: 

#  Alternate Method to ML.GENERATE_TEXT
## Extract Financial & ESG Data using AI.GENERATE

In [ ]:

print(f"🧠 Processing PDFs with {MODEL_NAME}...")
print(f"This analyzes {company1}, {company2}, and {company3} reports...")

generate_text_sql2 = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_RAW2_TABLE_ID}` AS
SELECT
  uri,
  AI.GENERATE(
    CONCAT(
      "You are an expert ESG and Financial analyst. Use only the information provided in the document to answer.\\n",
      "Fetch Financial and Sustainability Details including tabular and image data for each PDF.\\n",
      "Read the filename. If it is a sustainability report, please get only ESG details. If it is an annual report, get just the financials as below.\\n",
      "Ensure you fetch details so it has answers for the below: \\n",
      "Financial Performance\\n",
      "What is the total revenue this year?\\n",
      "How has revenue changed compared to last year?\\n",
      "What is the net profit or loss?\\n",
      "How have the earnings per share (EPS) changed?\\n",
      "Are gross margins improving or declining?\\n",
      "Are operating margins stable or volatile?\\n",
      "How has net margin shifted year over year?\\n",
      "Is cash flow from operations positive and consistent?\\n",
      "How much free cash flow is available?\\n",
      "How are working capital levels trending?\\n\\n",
      "Balance Sheet & Liquidity\\n",
      "What is the total debt level?\\n",
      "Is the debt-to-equity ratio rising or falling?\\n",
      "Does the company have enough liquidity to cover short-term liabilities?\\n",
      "What is the current ratio and quick ratio?\\n",
      "How much cash and equivalents are available?\\n",
      "Are interest expenses sustainable?\\n",
      "How sensitive is the company to interest rate changes?\\n",
      "What is the level of contingent liabilities?\\n\\n",
      "Business Strategy\\n",
      "What are the company’s core growth initiatives?\\n",
      "Is management investing in new markets or products?\\n",
      "How much is spent on research and development (R&D)?\\n",
      "Are acquisitions or divestitures planned?\\n",
      "Is the company expanding geographically?\\n",
      "What is the company’s competitive advantage?\\n",
      "Is the advantage sustainable against competitors?\\n",
      "Is digital transformation a core part of the strategy?\\n\\n",
      "Risk Factors\\n",
      "What macroeconomic risks affect the business?\\n",
      "What industry-specific risks are highlighted?\\n",
      "What operational risks are disclosed?\\n",
      "Are supply chain risks material?\\n",
      "Are legal or regulatory risks mentioned?\\n",
      "How is foreign exchange risk managed?\\n",
      "How dependent is revenue on a few customers?\\n",
      "Are raw material price risks significant?\\n\\n",
      "Governance & Leadership\\n",
      "Is the board majority independent?\\n",
      "How many women or minorities are on the board?\\n",
      "Is executive compensation performance-linked?\\n",
      "Does management own significant equity?\\n",
      "Are shareholder rights well protected?\\n",
      "Is succession planning disclosed?\\n",
      "Has management delivered on past promises?\\n",
      "Is the auditor independent and credible?\\n\\n",
      "Shareholder Value\\n",
      "Is the company paying dividends?\\n",
      "Is the dividend payout sustainable?\\n",
      "Is the dividend policy clear?\\n",
      "Are share buybacks taking place?\\n",
      "Is return on equity (ROE) improving?\\n",
      "Is return on assets (ROA) stable?\\n",
      "Has shareholder equity grown consistently?\\n",
      "What is the earnings guidance for next year?\\n\\n",
      "Investor Questions – Sustainability Reports\\n\\n",
      "Environmental\\n",
      "What is the total Scope 1 emissions?\\n",
      "What is the total Scope 2 emissions?\\n",
      "What is the total Scope 3 emissions?\\n",
      "Are science-based targets disclosed?\\n",
      "Has the company committed to net-zero?\\n",
      "What percentage of energy is renewable?\\n",
      "Is energy efficiency improving year over year?\\n",
      "What is the total water withdrawal?\\n",
      "What is water recycled or reused?\\n",
      "Is the company exposed to water stress?\\n",
      "How much total waste is generated?\\n",
      "How much waste is recycled or diverted from landfill?\\n",
      "Are hazardous waste levels disclosed?\\n",
      "How sustainable are raw material sourcing practices?\\n",
      "Are biodiversity risks addressed?\\n\\n",
      "Social\\n",
      "What is the total workforce size?\\n",
      "What percentage of employees are women?\\n",
      "What percentage of leadership roles are held by women?\\n",
      "What is minority representation in the workforce?\\n",
      "What is the employee turnover rate?\\n",
      "How many workplace injuries occurred?\\n",
      "What is the lost-time injury frequency rate?\\n",
      "Are labor rights respected across the supply chain?\\n",
      "Is child labor risk disclosed in supply chains?\\n",
      "Are employee training hours increasing per year?\\n",
      "Are wages above local living wage levels?\\n",
      "Is there employee engagement survey data?\\n",
      "What community development initiatives exist?\\n",
      "What philanthropic contributions were made?\\n",
      "Is there disclosure on data privacy and cybersecurity?\\n\\n",
      "Governance (ESG-specific)\\n",
      "Is there a dedicated board ESG committee?\\n",
      "How often does the board review ESG matters?\\n",
      "Is ESG risk included in enterprise risk management?\\n",
      "Is executive compensation tied to ESG goals?\\n",
      "Are ESG targets linked to management incentives?\\n",
      "Is ESG performance externally assured?\\n",
      "Are ESG disclosures aligned with GRI or SASB?\\n",
      "Does the company report under TCFD or ISSB?\\n",
      "Is CDP reporting publicly available?\\n",
      "Are stakeholder engagement practices disclosed?\\n\\n",
      "Risk & Opportunity\\n",
      "What are the identified physical climate risks?\\n",
      "What are the identified transition risks?\\n",
      "Is there disclosure on carbon pricing exposure?\\n",
      "How dependent is revenue on carbon-intensive activities?\\n",
      "What sustainability-linked opportunities are identified?\\n",
      "Are green products or services generating revenue?\\n",
      "Is access to sustainable finance increasing?\\n",
      "Are suppliers evaluated for ESG performance?\\n",
      "Are ESG clauses included in supplier contracts?\\n",
      "Is there disclosure on product lifecycle impacts?\\n",
      "Open and Read the PDF: ",
      JSON_VALUE(
        OBJ.GET_ACCESS_URL(
          OBJ.MAKE_REF(uri, '{QUALIFIED_CLOUD_RES_CONN}'),
          'r'
        ),
        '$.access_urls.read_url'
      )
    ),
    connection_id => '{QUALIFIED_CLOUD_RES_CONN}',
    endpoint => '{MODEL_ENDPOINT}',
    output_schema => 'uri STRING, response_text STRING'
  ).response_text AS response_text
FROM `{QUALIFIED_OBJ_TABLE_ID}`
WHERE uri LIKE '%{company1}%' OR uri LIKE '%{company2}%' OR uri LIKE '%{company3}%';
"""

try:
    print("⏳ Processing... This may take 2-3 minutes")
    job = client.query(generate_text_sql2)
    job.result()

    # Check results
    check_sql = f"SELECT COUNT(*) as processed_files FROM `{QUALIFIED_RAW2_TABLE_ID}`"
    result = client.query(check_sql).result()
    count = list(result)[0][0]
    print(f"✅ Successfully processed {count} files!")

except Exception as e:
    print(f"⚠️  Processing issue: {e}")


# 🧹 Extract Clean Text Response

In [ ]:

print("🧹 Cleaning and extracting response text...")

curate_results_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_CURATED_TABLE_ID}` AS
SELECT *,
  ARRAY_TO_STRING(
    ARRAY(
      SELECT JSON_VALUE(part, '$.text')
      FROM UNNEST(JSON_QUERY_ARRAY(
        JSON_QUERY(ml_generate_text_result, '$.candidates[0].content.parts')
      )) AS part
    ),
    ''
  ) AS response_text
FROM
  `{QUALIFIED_RAW_TABLE_ID}`;
"""

try:
    job = client.query(curate_results_sql)
    job.result()
    print("✅ Response text extracted and cleaned!")

    # Show sample of extracted text
    sample_sql = f"""
    SELECT uri, LEFT(response_text, 200) as sample_text
    FROM `{QUALIFIED_CURATED_TABLE_ID}`
    LIMIT 3
    """
    sample_df = bpd.read_gbq(sample_sql)
    print("\n📝 Sample extracted text:")
    for _, row in sample_df.iterrows():
        print(f"File: {row['uri']}")
        print(f"Text: {row['sample_text']}...\n")

except Exception as e:
    print(f"⚠️  Text extraction issue: {e}")

# 📊 Extract Structured Financial & ESG Metrics

In [ ]:

print("📊 Generating structured financial and ESG metrics...")

generate_table_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_FINAL_TABLE_ID}` AS
SELECT *,
    DATE(CAST(fiscal_year AS STRING) || "-01-01") AS fiscal_year_date,
    CASE
        WHEN uri LIKE '%{company1}%' THEN '{company1}'
        WHEN uri LIKE '%{company2}%' THEN '{company2}'
        WHEN uri LIKE '%{company3}%' THEN '{company3}'
    END as company_name_formatted
FROM (
    SELECT *
    FROM AI.GENERATE_TABLE(
        MODEL `{QUALIFIED_MODEL_ID}`,
        (
            SELECT CONCAT(
                "You are an expert financial analyst and ESG/Climate analyst. Extract all financial metrics and sustainability metrics and KPIs from the provided text into structured data. ",
                "For Annual reports, focus on getting the financial metrics. And for Sustainability reports focus on getting ESG metrics. ",
                "Normalize numbers by removing commas, spaces, or symbols. ",
                "Only include actual numerical financial data. ",
                "TEXT: ",
                response_text
            ) AS prompt,
            response_text as extracted_text,
            uri AS uri
        FROM `{QUALIFIED_CURATED_TABLE_ID}`
        ),
        STRUCT(
            'uris STRING, report_type STRING, company_name STRING, fiscal_year INT64, revenue_millions FLOAT64, net_income_millions FLOAT64, total_assets_millions FLOAT64, total_liabilities_millions FLOAT64, equity_millions FLOAT64, eps_basic FLOAT64, eps_diluted FLOAT64, dividends_per_share FLOAT64, employee_count INT64, business_segments ARRAY<STRING>, total_ghg_emissions FLOAT64, scope_1_ghg_emissions FLOAT64, scope_2_ghg_emissions FLOAT64, scope_3_ghg_emissions FLOAT64, carbon_intensity FLOAT64, energy_consumption FLOAT64, renewable_energy_usage FLOAT64, water_consumption FLOAT64, waste_generated FLOAT64, waste_recycled FLOAT64, sustainable_financing FLOAT64, reporting_period STRING, page_references ARRAY<STRING>, contexts ARRAY<STRING>, verbatim_quotes ARRAY<STRING>'
            AS output_schema,
            8192 AS max_output_tokens,
            0 AS temperature
        )
    ) t
);
"""

try:
    print("⏳ Structuring data... This may take 3-4 minutes")
    job = client.query(generate_table_sql)
    job.result()
    print("✅ Structured metrics extracted successfully!")

except Exception as e:
    print(f"⚠️  Table generation issue: {e}")

# 🔮 Revenue Forecasting with TimesFM 2.0

In [ ]:

print("🔮 Generating revenue forecasts with TimesFM 2.0...")

forecast_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_FORECAST_TABLE_ID}` AS
SELECT
  *
FROM
  AI.FORECAST(
    TABLE `{QUALIFIED_FINAL_TABLE_ID}`,
    data_col => 'revenue_millions',
    timestamp_col => 'fiscal_year_date',
    model => 'TimesFM 2.0',
    id_cols => ['company_name_formatted'],
    horizon => 5,
    confidence_level => .95
  )
"""

print("⏳ Forecasting...")
job = client.query(forecast_sql)
job.result()
print("✅ Forecast table created successfully!")

In [ ]:
# import requests
# from google.cloud import bigquery

# PROJECT_ID = "intellitrend-project-dev"   # update if needed
# DATASET_ID = "db_reports_insights_annual_esg"
# TABLE_ID = "all_reports_annual_esg"

# # Step 0: Fetch file list from GitHub
# url = "https://api.github.com/repos/intellitrend-global/google_hackathon_bq_ai/contents/annual_esg_reports"
# response = requests.get(url)
# files = response.json()

# pdf_files = [(f["name"], f["download_url"]) for f in files if f["name"].endswith(".pdf")]

# print("📄 Found PDF files on GitHub:")
# for name, link in pdf_files:
#     print(f"   {name} -> {link}")

# # Step 1: Initialize BigQuery client
# client = bigquery.Client(project=PROJECT_ID)

# # Step 2: Ensure dataset exists (create if not)
# dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_ID)
# try:
#     client.get_dataset(dataset_ref)  # Check if it exists
#     print(f"✅ Dataset {DATASET_ID} already exists")
# except Exception:
#     dataset = bigquery.Dataset(dataset_ref)
#     dataset.location = "US"
#     client.create_dataset(dataset, timeout=30)
#     print(f"📦 Created dataset {DATASET_ID}")

# # Step 3: Define schema
# schema = [
#     bigquery.SchemaField("filename", "STRING"),
#     bigquery.SchemaField("uri", "STRING"),
# ]

# # Step 4: Prepare rows
# rows = [{"filename": name, "uri": link} for name, link in pdf_files]

# # Step 5: Load into BigQuery
# job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
# table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# job = client.load_table_from_json(rows, table_ref, job_config=job_config)
# job.result()

# print(f"✅ Table {table_ref} created/updated with {len(pdf_files)} GitHub files")
